- [TCPServer实现](#TCPServer实现)
  * [TCPServer class](#TCPServer-class)
  * [BaseRequestHandler class](#BaseRequestHandler-class)
  * [StreamRequestHandler class](#StreamRequestHandler-class)
  * [ThreadingMixin and ForkingMixin class](#ThreadingMixin-and-ForkingMixin-class)
  * [Summary](#Summary)
  * [Reference](#Reference)

# TCPServer实现

在python lib中，socketserver.py里实现了简单的TCP server。在该实现中，作者将一个tcp server抽象为两部分：server与handler。其中，server负责
监听服务器端口，处理socket连接等。而handler负责根据连接的请求内容返回请求结果。

## TCPServer class

我们知道，HTTP是应用层的协议，其传输层协议是TCP协议，而TCP协议又是通过socket来实现的。因此，首先学习如何用socket实现一个TCP协议的server。

以python lib中socketserver.py里的TCPServer class为例，介绍如何实现一个TCP协议的server。

在介绍如何实现之前，先看看TCPServer的用法，以便有个大致了解。
```Python
import socketserver

class MyTCPHandler(socketserver.BaseRequestHandler):
    """
    The request handler class for our server.

    It is instantiated once per connection to the server, and must
    override the handle() method to implement communication to the
    client.
    """

    def handle(self):
        # self.request is the TCP socket connected to the client
        self.data = self.request.recv(1024).strip()
        print("{} wrote:".format(self.client_address[0]))
        print(self.data)
        # just send back the same data, but upper-cased
        self.request.sendall(self.data.upper())

if __name__ == "__main__":
    HOST, PORT = "localhost", 9999

    # Create the server, binding to localhost on port 9999
    server = socketserver.TCPServer((HOST, PORT), MyTCPHandler)
    # Activate the server; this will keep running until you
    # interrupt the program with Ctrl-C
    server.serve_forever(1)
    
###################### Client side ######################
import socket
import sys

HOST, PORT = "localhost", 9999
data = " ".join(sys.argv[1:])

for i in range(10):
    # Create a socket (SOCK_STREAM means a TCP socket)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        # Connect to server and send data
        sock.connect((HOST, PORT))
        sock.sendall(bytes(data + "\n", "utf-8"))

        # Receive data from the server and shut down
        received = str(sock.recv(1024), "utf-8")

        print("Sent:     {}".format(data))
        print("Received: {}".format(received))
```
在server端，先传入host和port以及handler创建一个TCPServer实例，然后执行serve_forever一直等待处理request。

host和port不需要解释，handler类必须有handle函数，直接处理request的逻辑就写在该函数内。

__一个TCPServer实例化__大致就是实例化socket并监听端口的过程，即创建一个socket实例，并让该socket监听传入的port端口。

```Python
self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
self.socket.bind(self.server_address)
self.socket.listen(self.request_queue_size)
```

__server_forever的逻辑__大致如下：
1. 通过select/poll监听socket的read事件，当socket状态变为可读时（即有client连接到server），开始处理socket连接。
2. 处理socket连接时，首先判断request是否合法，然后调用handler来处理socket连接，处理完成后关闭socket连接。

```Python
def _handle_request_noblock(self):
    """Handle one request, without blocking.

    I assume that selector.select() has returned that the socket is
    readable before this function was called, so there should be no risk of
    blocking in get_request().
    """
    try:
        request, client_address = self.get_request()
    except OSError:
        return
    if self.verify_request(request, client_address):
        try:
            self.process_request(request, client_address)
        except Exception:
            self.handle_error(request, client_address)
            self.shutdown_request(request)
        except:
            self.shutdown_request(request)
            raise
    else:
        self.shutdown_request(request)
            
with _ServerSelector() as selector:
    selector.register(self, selectors.EVENT_READ)

    while not self.__shutdown_request:
        ready = selector.select(poll_interval)
        if ready:
            self._handle_request_noblock()
        self.service_actions()

```

## BaseRequestHandler class
前面说过Handler是要传给TCPServer的第二个参数，作用是真正处理request。

BaseRequestHandler是其他Handler的基类，它的`__init__`函数规定了一个handler必须依次调用如下函数：
1. `setup()`: 调用handle前需要作的一些准备工作。
2. `handle()`：真正处理request的函数。
3. `finish()`：一些后处理工作。

## StreamRequestHandler class
StreamRequestHandler继承BaseRequestHandler，它override了`setup()`和`finish()`：
1. `setup()`: 通过调用`socket.makefile`将socket连接用文件读写的方式来读写，即通过rfile来read，wfile来write，方便使用。
2. `finish()`：关闭rfile和wfile。

## ThreadingMixin and ForkingMixin class
可以发现，TCPServer串行调用Handler来处理request，如果Handler处理速度特别慢，就会造成server的效率很低（在handle函数中sleep试试~）。因此，ThreadingMixin class通过多线程调用handle来提高服务器处理请求的效率。

```Python
def process_request(self, request, client_address):
    """Start a new thread to process the request."""
    t = threading.Thread(target = self.process_request_thread,
                         args = (request, client_address))
    t.daemon = self.daemon_threads
    t.start()
```

类似地，ForkingMixIn class通过fork子进程来处理request。

## Summary
TCPServer的本质是socket，即通过socket来处理tcp连接。值得注意的地方有两点：
1. 用I/O multiplexing来提高cpu利用率。
2. Handler类是真正处理request的类。

## Reference
* [Difference of select/poll/epoll](https://www.ulduzsoft.com/2014/01/select-poll-epoll-practical-difference-for-system-architects/#Polling_with_epoll)